## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
import sys
sys.path.append("..")
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Pizarro,CO,4.95,-77.37,84.22,73,23,8.23,few clouds
1,1,Tuktoyaktuk,CA,69.45,-133.04,50.00,71,90,18.34,overcast clouds
2,2,Mataura,NZ,-46.19,168.86,47.17,95,72,2.10,broken clouds
3,3,Mehamn,NO,71.04,27.85,50.00,87,75,9.17,broken clouds
4,4,Nome,US,64.50,-165.41,55.40,76,1,3.36,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 105


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Pizarro,CO,4.95,-77.37,84.22,73,23,8.23,few clouds
5,5,Saint George,US,37.10,-113.58,100.99,15,1,3.36,clear sky
6,6,Puerto Leguizamo,CO,-0.19,-74.78,85.24,61,38,2.10,light rain
10,10,Rikitea,PF,-23.12,-134.97,72.68,76,57,12.37,broken clouds
11,11,Palmdale,US,34.58,-118.12,96.80,8,1,14.99,clear sky
12,12,Rapid Valley,US,44.06,-103.15,77.00,38,1,8.05,clear sky
13,13,Sibu,MY,2.30,111.82,77.00,88,40,6.93,scattered clouds
14,14,Dingle,PH,11.00,122.67,76.66,83,100,4.14,overcast clouds
15,15,Jamestown,US,42.10,-79.24,73.99,78,90,14.99,overcast clouds
19,19,Ribeira Grande,PT,38.52,-28.70,75.20,69,20,9.17,few clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                385
City                   385
Country                384
Lat                    385
Lng                    385
Max Temp               385
Humidity               385
Cloudiness             385
Wind Speed             385
Current Description    385
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Pizarro,CO,4.95,-77.37,84.22,73,23,8.23,few clouds
5,5,Saint George,US,37.10,-113.58,100.99,15,1,3.36,clear sky
6,6,Puerto Leguizamo,CO,-0.19,-74.78,85.24,61,38,2.10,light rain
10,10,Rikitea,PF,-23.12,-134.97,72.68,76,57,12.37,broken clouds
11,11,Palmdale,US,34.58,-118.12,96.80,8,1,14.99,clear sky


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Pizarro,CO,84.22,few clouds,4.95,-77.37,
5,Saint George,US,100.99,clear sky,37.10,-113.58,
6,Puerto Leguizamo,CO,85.24,light rain,-0.19,-74.78,
10,Rikitea,PF,72.68,broken clouds,-23.12,-134.97,
11,Palmdale,US,96.80,clear sky,34.58,-118.12,
12,Rapid Valley,US,77.00,clear sky,44.06,-103.15,
13,Sibu,MY,77.00,scattered clouds,2.30,111.82,
14,Dingle,PH,76.66,overcast clouds,11.00,122.67,
15,Jamestown,US,73.99,overcast clouds,42.10,-79.24,
19,Ribeira Grande,PT,75.20,few clouds,38.52,-28.70,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Pizarro,CO,84.22,few clouds,4.95,-77.37,"Playa Pizarro, Bajo Baudó, Chocó, Colombia"
5,Saint George,US,100.99,clear sky,37.10,-113.58,Best Western Plus Abbey Inn
6,Puerto Leguizamo,CO,85.24,light rain,-0.19,-74.78,la casona de juancho
10,Rikitea,PF,72.68,broken clouds,-23.12,-134.97,Pension Maro'i
11,Palmdale,US,96.80,clear sky,34.58,-118.12,Courtyard by Marriott Palmdale
...,...,...,...,...,...,...,...
697,Marrakesh,MA,100.40,scattered clouds,31.63,-8.01,Les Jardins de La Koutoubia
698,Taytayan,PH,76.73,light rain,7.73,126.50,Villa Dominga
699,Keti Bandar,PK,87.03,overcast clouds,24.14,67.45,Haji laloo memon house
700,Beidao,CN,70.09,scattered clouds,34.57,105.89,Shell Tianshui Maiji District Erma Road Walkin...


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))